# Meander Maker

If you haven't ever used the `googlemaps` python package, there are a few small setup steps to take first. You'll need to install it by running the next cell. Additionally, we will use `gmplot` to render the outputs and `polyline` to decode [Google's "Polyline"](https://developers.google.com/maps/documentation/utilities/polylineutility) hash. Finally, generate an API key to interface with Google's services. (There is a free tier, but if you use it a lot, they will start charging you for access.)

In [ ]:
!pip install googlemaps
!pip install gmplot
!pip install polyline
!pip install haversine

If you would like to tinker around with the code, you'll need your own API key for Google Maps. You can register one from this address: [https://developers.google.com/places/web-service/get-api-key](https://developers.google.com/places/web-service/get-api-key). Once you have one, create a file in the directory containing this notebook called `.secret.key` and simply paste in your newly generated API key, no quotes or line breaks or anything. Save, exit, and you should be good to go.

In [1]:
import goplaces as gp

%load_ext autoreload
%autoreload 2

Let's decide which topic to explore:

In [2]:
topic = gp.get_topic()

What theme walk would you like to explore today? good views


In [ ]:
loc = gp.get_loc(current=False)

[autoreload of goplaces failed: Traceback (most recent call last):
  File "/anaconda3/envs/learn-env/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/anaconda3/envs/learn-env/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 368, in superreload
    module = reload(module)
  File "/anaconda3/envs/learn-env/lib/python3.7/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/anaconda3/envs/learn-env/lib/python3.7/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 630, in _exec
  File "<frozen importlib._bootstrap_external>", line 724, in exec_module
  File "<frozen importlib._bootstrap_external>", line 860, in get_code
  File "<frozen importlib._bootstrap_external>", line 791, in source_to_code
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/Users/dakaspa

In [ ]:
one_way_json = gp.build_list(loc, topic)
dest_list = gp.lat_lng_list(one_way_json)
walk = gp.walk(dest_list, verbose=True)

In [ ]:
gp.html_builder(loc, walk, tab=True)

---------TESTING-----------